In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Flatten


In [ ]:
print(tf.__version__)

2.6.0


### Prepare Dataset

In [ ]:
mnist = tf.keras.datasets.mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
# add a channels dimention

X_train = X_train[..., tf.newaxis].astype("float32")
X_test = X_test[..., tf.newaxis].astype("float32")

In [ ]:
X_train.shape

(60000, 28, 28, 1)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(10000).batch(32)
test_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(32)

### Define Model

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, num_class):
    super().__init__()
    self.conv1 = Conv2D(32, (3, 3), activation='relu')
    self.flatten = Flatten()
    self.fc1 = Dense(128, activation='relu')
    self.fc2 = Dense(num_class)

  def call(self, x):
    y = self.conv1(x)
    # y.shape = 26*26*32

    w = self.flatten(y)
    # w.shape = 1 * 21632

    z = self.fc1(w)
    # z.shape = 1 * 128

    output = self.fc2(z)
    # output.shape = 1 * 10

    return output

model = MyModel(10)

In [ ]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

In [ ]:
def train_step(images, y):
  with tf.GradientTape() as tape:
    y_pred =  model(images, training = True)
    loss = loss_function(y, y_pred)

  # calculate gradient
  gradient = tape.gradient(loss, model.trainable_variables)
  
  # update
  optimizer.apply_gradients(zip(gradient, model.trainable_variables))

  train_loss(loss)
  train_acc(y, y_pred)


In [ ]:
def test_step(images, y):
  y_pred = model(images, training=False)
  loss = loss_function(y, y_pred)

  test_loss(loss)
  test_acc(y, y_pred)

In [ ]:
from tqdm import tqdm

In [ ]:
def train():
  epochs = 5

  for epoch in range(epochs):
    train_loss.reset_state()
    train_acc.reset_state()
    test_loss.reset_state()
    test_acc.reset_state()

    for images, labels in tqdm(train_data):
      train_step(images, labels)

    for images, labels in tqdm(test_data):
      test_step(images, labels)

    print('epoch: ', epoch + 1)
    print('loss: ', train_loss.result())
    print('accuracy: ', train_acc.result())

    print('epoch: ', epoch + 1)
    print('val loss: ', test_loss.result())
    print('val accuracy: ', test_acc.result())

In [ ]:
# fit

train()

100%|██████████| 313/313 [00:02<00:00, 118.57it/s]


epoch:  1
loss:  tf.Tensor(0.0071229893, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.9975833, shape=(), dtype=float32)
epoch:  1
val loss:  tf.Tensor(0.071983896, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.9834, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 121.15it/s]


epoch:  2
loss:  tf.Tensor(0.004599823, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.9985, shape=(), dtype=float32)
epoch:  2
val loss:  tf.Tensor(0.06381927, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.9849, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 120.27it/s]


epoch:  3
loss:  tf.Tensor(0.0057296385, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.9982833, shape=(), dtype=float32)
epoch:  3
val loss:  tf.Tensor(0.07149253, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.985, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 121.81it/s]


epoch:  4
loss:  tf.Tensor(0.0044878284, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.9985167, shape=(), dtype=float32)
epoch:  4
val loss:  tf.Tensor(0.085839376, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.9838, shape=(), dtype=float32)


100%|██████████| 313/313 [00:02<00:00, 122.21it/s]

epoch:  5
loss:  tf.Tensor(0.003135906, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.9989333, shape=(), dtype=float32)
epoch:  5
val loss:  tf.Tensor(0.07076751, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.9853, shape=(), dtype=float32)
